In [121]:
import pandas as pd
from prophet import Prophet
import plotly.express as px
import requests
from datetime import datetime 
import json

In [98]:
##consumiendo APIIIIII
# URL de la API
url = "https://www.datos.gov.co/resource/32sa-8pi3.json"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("ocurrio un error", response.status_code)

[{'valor': '4047.11', 'unidad': 'COP', 'vigenciadesde': '2023-11-03T00:00:00.000', 'vigenciahasta': '2023-11-03T00:00:00.000'}, {'valor': '4117.71', 'unidad': 'COP', 'vigenciadesde': '2023-11-02T00:00:00.000', 'vigenciahasta': '2023-11-02T00:00:00.000'}, {'valor': '4114.29', 'unidad': 'COP', 'vigenciadesde': '2023-11-01T00:00:00.000', 'vigenciahasta': '2023-11-01T00:00:00.000'}, {'valor': '4060.83', 'unidad': 'COP', 'vigenciadesde': '2023-10-31T00:00:00.000', 'vigenciahasta': '2023-10-31T00:00:00.000'}, {'valor': '4120.62', 'unidad': 'COP', 'vigenciadesde': '2023-10-28T00:00:00.000', 'vigenciahasta': '2023-10-30T00:00:00.000'}, {'valor': '4154.94', 'unidad': 'COP', 'vigenciadesde': '2023-10-27T00:00:00.000', 'vigenciahasta': '2023-10-27T00:00:00.000'}, {'valor': '4201.53', 'unidad': 'COP', 'vigenciadesde': '2023-10-26T00:00:00.000', 'vigenciahasta': '2023-10-26T00:00:00.000'}, {'valor': '4213.97', 'unidad': 'COP', 'vigenciadesde': '2023-10-25T00:00:00.000', 'vigenciahasta': '2023-10-25

In [99]:
df = pd.DataFrame(data)
print(df['vigenciadesde'])



0      2023-11-03T00:00:00.000
1      2023-11-02T00:00:00.000
2      2023-11-01T00:00:00.000
3      2023-10-31T00:00:00.000
4      2023-10-28T00:00:00.000
                ...           
995    2019-08-29T00:00:00.000
996    2019-08-28T00:00:00.000
997    2019-08-27T00:00:00.000
998    2019-08-24T00:00:00.000
999    2019-08-23T00:00:00.000
Name: vigenciadesde, Length: 1000, dtype: object


In [100]:
df.shape
df.isnull().sum()

valor            0
unidad           0
vigenciadesde    0
vigenciahasta    0
dtype: int64

In [105]:
df['fecha'] = pd.to_datetime(df['vigenciadesde']).dt.strftime("%y%m%d")
df['ds'] = pd.to_datetime(df['fecha'], format='%y%m%d')
print(df['ds'])
print(df['ds'][9])
df['y'] = pd.to_numeric(df['valor'])
print(df['y'])


0     2023-11-03
1     2023-11-02
2     2023-11-01
3     2023-10-31
4     2023-10-28
         ...    
995   2019-08-29
996   2019-08-28
997   2019-08-27
998   2019-08-24
999   2019-08-23
Name: ds, Length: 1000, dtype: datetime64[ns]
2023-10-21 00:00:00
0      4047.11
1      4117.71
2      4114.29
3      4060.83
4      4120.62
        ...   
995    3477.53
996    3457.89
997    3432.85
998    3399.95
999    3376.99
Name: y, Length: 1000, dtype: float64


In [114]:
px.line(df, x='ds' , y='y' ,title='Dollar')

In [107]:
# print(df)
df = df.iloc[:, 2:]
print(df)

               vigenciadesde            vigenciahasta   fecha        y  \
0    2023-11-03T00:00:00.000  2023-11-03T00:00:00.000  231103  4047.11   
1    2023-11-02T00:00:00.000  2023-11-02T00:00:00.000  231102  4117.71   
2    2023-11-01T00:00:00.000  2023-11-01T00:00:00.000  231101  4114.29   
3    2023-10-31T00:00:00.000  2023-10-31T00:00:00.000  231031  4060.83   
4    2023-10-28T00:00:00.000  2023-10-30T00:00:00.000  231028  4120.62   
..                       ...                      ...     ...      ...   
995  2019-08-29T00:00:00.000  2019-08-29T00:00:00.000  190829  3477.53   
996  2019-08-28T00:00:00.000  2019-08-28T00:00:00.000  190828  3457.89   
997  2019-08-27T00:00:00.000  2019-08-27T00:00:00.000  190827  3432.85   
998  2019-08-24T00:00:00.000  2019-08-26T00:00:00.000  190824  3399.95   
999  2019-08-23T00:00:00.000  2019-08-23T00:00:00.000  190823  3376.99   

            ds  
0   2023-11-03  
1   2023-11-02  
2   2023-11-01  
3   2023-10-31  
4   2023-10-28  
..       

In [108]:
m=Prophet()
m.fit(df)

17:26:03 - cmdstanpy - INFO - Chain [1] start processing
17:26:03 - cmdstanpy - INFO - Chain [1] done processing


In [117]:
future=m.make_future_dataframe(periods=2, freq='D')
future.tail()

,ds
997,2023-11-01
998,2023-11-02
999,2023-11-03
1000,2023-11-04
1001,2023-11-05


In [ ]:
forecast=m.predict(future)
forecast[['ds','yhat','yhat_lower', 'yhat_upper' ]]

In [123]:
url= "http://localhost:4000/prediccion"
forecast_filtered = forecast[forecast['ds'] >= datetime(2023, 1, 1)]
args= forecast_filtered[['ds','yhat','yhat_lower', 'yhat_upper' ]]
data = {"datos": args.astype({'ds': 'str'}).to_dict(orient='records')}

print("Enviar:", data)

response = requests.post(url=url, json=data)

print("status res:",response.status_code)
print("respuesta servidor:", response.text)

Enviar: {'datos': [{'ds': '2023-01-04', 'yhat': 4829.641512191873, 'yhat_lower': 4702.283692973834, 'yhat_upper': 4954.132302467092}, {'ds': '2023-01-05', 'yhat': 4824.421078265801, 'yhat_lower': 4698.251297472613, 'yhat_upper': 4960.582529365872}, {'ds': '2023-01-06', 'yhat': 4822.464641548895, 'yhat_lower': 4695.8982478147445, 'yhat_upper': 4951.535315275659}, {'ds': '2023-01-07', 'yhat': 4814.477744535048, 'yhat_lower': 4687.157364347556, 'yhat_upper': 4947.849561402858}, {'ds': '2023-01-11', 'yhat': 4796.082410491749, 'yhat_lower': 4666.376230141903, 'yhat_upper': 4925.434458608589}, {'ds': '2023-01-12', 'yhat': 4787.422533352483, 'yhat_lower': 4666.663277335248, 'yhat_upper': 4910.411851353462}, {'ds': '2023-01-13', 'yhat': 4782.314146921719, 'yhat_lower': 4657.327646621018, 'yhat_upper': 4913.071503356916}, {'ds': '2023-01-14', 'yhat': 4771.559982175262, 'yhat_lower': 4646.398259077551, 'yhat_upper': 4887.896689466893}, {'ds': '2023-01-18', 'yhat': 4747.519448938442, 'yhat_lower'